<a href="https://colab.research.google.com/github/ML-Bioinfo-CEITEC/ECCB2022/blob/main/notebooks/00_Genomic_benchmarks_HF_export.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installation

In [1]:
!pip install -qq transformers genomic-benchmarks datasets

     |████████████████████████████████| 4.7 MB 14.6 MB/s 
     |████████████████████████████████| 365 kB 55.3 MB/s 
     |████████████████████████████████| 6.6 MB 35.8 MB/s 
     |████████████████████████████████| 120 kB 64.0 MB/s 
     |████████████████████████████████| 2.3 MB 50.0 MB/s 
     |████████████████████████████████| 115 kB 60.4 MB/s 
     |████████████████████████████████| 212 kB 67.2 MB/s 
     |████████████████████████████████| 127 kB 67.5 MB/s 


In [13]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


## Human non-tata promoters



In [2]:
from genomic_benchmarks.loc2seq import download_dataset

download_dataset("human_nontata_promoters", version=0)

/usr/local/lib/python3.7/dist-packages/genomic_benchmarks/utils/datasets.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
Downloading...
From: https://drive.google.com/uc?id=1VdUg0Zu8yfLS6QesBXwGz1PIQrTW3Ze4
To: /root/.genomic_benchmarks/human_nontata_promoters.zip
100%|██████████| 11.8M/11.8M [00:00<00:00, 45.5MB/s]


PosixPath('/root/.genomic_benchmarks/human_nontata_promoters')

In [3]:
from pathlib import Path

tmp_dict = {}

for dset in ['train', 'test']:
  for c in ['negative', 'positive']:
    for f in Path(f'/root/.genomic_benchmarks/human_nontata_promoters/{dset}/{c}/').glob('*.txt'):
      txt = f.read_text()
      tmp_dict[f.stem] = (dset, int(c == "positive"), txt)

In [5]:
import pandas as pd

df = pd.DataFrame.from_dict(tmp_dict).T.rename(columns = {0: "dset", 1: "labels", 2: "seq"})
#df.to_pickle("human_nontata_promoters.pkl")
df

,dset,labels,seq
15928,train,0,ACAGATTCAGGATGTCCTGTCGGGGCATGGACCCTGGAAAGCTGCG...
14132,train,0,ATGATTTTAATACCACATTGGTAGACCTGCAATCCCCCAGAGAAGG...
9997,train,0,CCCCGGGATGCCCCTAGCCCCTCCCTGTGAGCTGCCTCTCACAGGT...
7572,train,0,GACCACCAGGGTGAGCTGGGGGTGGGCGGGGCCTGCCTGGCCAGGG...
10834,train,0,TTTTCCAGAAAATGAGGTGTGGCCAAACATCTTCAGGCTTTTCCTT...
...,...,...,...
FP013432,test,1,CCTGTAGCCTTTAAAAGTAGACATCTTTTTCCAACCAAGTTTAGGC...
FP008703,test,1,GTTAAATACGCTGGCGGACCGTGTGCACTGCGCCAACGGGCCGTGT...
FP003964,test,1,CCATCACACAGCAAGGCTAGCCAGATGCCCCTCACTCTTCTCGGGC...
FP007349,test,1,GCGTCTGAGAAGCAGAAGCCACACGTCCCGGCGACCTAAGTTCTGG...


In [11]:
from datasets import Dataset, DatasetDict

ds = Dataset.from_pandas(df)
ds[0]

{'dset': 'train',
 'labels': 0,
 'seq': 'ACAGATTCAGGATGTCCTGTCGGGGCATGGACCCTGGAAAGCTGCGGACACCAGGAGGGCAGGCAAGAGAGTCTCATCTCTTGCTCCCTAGGAGCTATGAGTTGAGGGCGCCGTCTGAGCAGGAGGGACGGACGGGTGCCCAGGGTTTGAGGAAAGAGGGGTGTGGGAAGGACGCATGCTAGAACTTCAGAGCAGTTCAGCAGGTGCAGAATGGGAGTTATCATGGGGACTGTGGGAGAAGGGGCGGTGGG',
 '__index_level_0__': '15928'}

In [12]:
dds = DatasetDict({
    'train': ds.filter(lambda x: x["dset"] == "train").remove_columns(['__index_level_0__', 'dset']),
    'test':  ds.filter(lambda x: x["dset"] == "test").remove_columns(['__index_level_0__', 'dset'])
})

dds

  0%|          | 0/37 [00:00<?, ?ba/s]

  0%|          | 0/37 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['labels', 'seq'],
        num_rows: 27097
    })
    test: Dataset({
        features: ['labels', 'seq'],
        num_rows: 9034
    })
})

In [14]:
dds.push_to_hub("human_nontata_promoters")

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/huggingface_hub/hf_api.py:2168: FutureWarning: `identical_ok` has no effect and is deprecated. It will be removed in 0.11.0.
  FutureWarning,


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]